In [ ]:
# Установка пакетов
!pip install wikipedia
!pip install natasha
!pip install stop-words

In [ ]:
# Импорт библиотек
# Библиотека для работы с Википедией
import wikipedia
# Библиотека для работы с другими источниками
import requests
# Библиотека для создания регулярных выражений
import re
# Библиотека для визуализации облака слов
import matplotlib.pyplot as plt
%matplotlib inline
# Библиотеки для NLP
import natasha as nt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# Библиотека для генерации облака слов
from wordcloud import WordCloud
# Библиотека со списком стоп-слов для облака
from stop_words import get_stop_words
# Установка пакетов для NLP
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('omw-1.4')

In [3]:
# Запись исходного и обработаного текста в файл
def write_text_in_file(text, raw_text = 1):
    if raw_text:
        with open('Исходный текст.txt', 'w') as write_raw_text:
            write_raw_text.write(text)
    else:
        with open(f'Обработанный текст.txt', 'w') as write_processed_text:
            write_processed_text.write(text)

In [4]:
# Загрузка содержимого страницы Википедии или текстового файла
def upload_text(language, source_url, source_name = 'wiki'):

    # Получение текстовых данных из источника
    if source_name == 'wiki':
        wikipedia.set_lang(language)
        wiki = wikipedia.page(source_url)

        # Извлечение текста из полученной страницы
        text = wiki.content

        # Вызов функции очищения текста
        clean_text(text)

        # Запись сырого текста в файл
        write_text_in_file(text, raw_text = 1)

    # Получение текстовых данных из источника
    elif source_name == 'txt':
        response = requests.get(source_url)
        text = response.text

        # Вызов функции очищения текста
        clean_text(text)

        # Запись сырого текста в файл
        write_text_in_file(text, raw_text = 1)

In [5]:
# Очищение текста от специальных символов, англ. слов,
# разделителей слов, сокращений и переносов
def clean_text(text):

    # Удаление переносов строк
    character_map = {
    ord('\n') : ' ',
    ord('\t') : ' ',
    ord('\r') : ' ',
    ord('ё')  : 'е'
    }
    text = text.translate(character_map)

    # Удаление специальных символов
    text = text.strip() # удаление пробелов в начале и конце строки
    text = re.sub(r'-|\\', ' ', text) # разделение слов через дефис или слэш
    text = re.sub(r'[^\w\s]+|[\d]+', '', text) # удаление спец. символов
    text = re.sub(r'[^а-яА-ЯёЁ\d\s]', '', text) # удаление слов на английском языке, опционально
    text = re.sub(r'\b\w{,2}\b', '', text) # убираем слова из 2-х букв и меньше не из стоп-слов

    # Вызов функции обработки текста
    nlp_russian(text)

In [6]:
# Обработка русскоязычного текста 
def nlp_russian(text):

    # Инициализация объектов для преобразований и анализа текста
    morph_vocab = nt.MorphVocab()
    segmenter = nt.Segmenter()
    emb = nt.NewsEmbedding()
    morph_tagger = nt.NewsMorphTagger(emb)
    names_extractor = nt.NamesExtractor(morph_vocab)
    syntax_parser = nt.NewsSyntaxParser(emb)

    # Инициализация объекта из текста для преобразований
    doc = nt.Doc(text)

    # Токенизация и морфологический анализ токенов
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)

    # Создание словаря терминов
    stop_words = set(stopwords.words('russian'))

    # Лемматизация слов
    text_processed = ''
    for token in doc.tokens:
            if token.lemma not in stop_words: # and token.pos == 'Noun': - условие для облака понятий
                token.lemmatize(morph_vocab)
                text_processed = text_processed + " " + token.lemma

    # Запись обработанного текста в файл
    write_text_in_file(text_processed, raw_text = 0)

    # Вызов функции генерации облака слов
    wordcloud_generation(text_processed)

In [7]:
# Настройка визуализации облака слов
def plot_cloud(wordcloud):
    plt.figure(figsize=(40, 30))
    plt.imshow(wordcloud) 
    plt.axis("off")

In [8]:
# Записываем в переменную стоп-слова русского языка
def wordcloud_generation(text):
    STOPWORDS_RU = get_stop_words('russian')
    # Генерируем облако слов
    wordcloud = WordCloud(width = 2000, 
                      height = 1500, 
                      #random_state=1,
                      background_color='grey', 
                      margin=20, 
                      colormap='Pastel2', 
                      collocations=False, # учет коллокаций
                      stopwords = STOPWORDS_RU # доп. фильтрация стоп-слов
                      ).generate(text)
    # Визуализируем облако
    plot_cloud(wordcloud)

In [9]:
# Кастомные классы исключений
class WrongSourceName(ValueError):
    pass

class WrongSourceURL(ValueError):
    pass

In [10]:
# Обработка ввода названия источника текста, ошибка если его нет в списке источников
def validate_source_name(source_name, sources):
    if source_name not in sources:
        raise WrongSourceName(source_name)

In [11]:
# Обработка ввода страницы с txt-файлом, если у нее неверный формат, то ошибка
def validate_source_url(source_url, source_name):
    if source_name == 'txt' and (not re.search('\.txt', source_url) 
                    or not re.match('http:\/\/www\.', source_url)):
        raise WrongSourceURL(source_url)

In [ ]:
# Выбираем источник текста и его адрес/название статьи Википедии
language = 'ru'
sources = ['wiki', 'txt']

source_name = input('Введите тип источника - "wiki" для Википедии или "txt" для страницы с файлом и нажмите Enter: ')
validate_source_name(source_name, sources)

source_url = input('Введите название статьи Википедии или страницы с файлом формата .txt и нажмите Enter: ')
validate_source_url(source_url, source_name)

text = upload_text(language, source_url, source_name)